# Classificazione di 5 classi di stelle per mezzo di curve di luce simulate
Progetto per l'esame del corso *Machine Learning per la fisica* A.A. 2023-24 Prof. Donato Cascio\
Mario Lauriano (matr. 0767930) - LM Fisica

Tale progetto si inserisce all'interno di una campagna osservativa in corso di stelle supergiganti rosse (RSG) guidata dal Dott. Fabrizio Bocchino di INAF-Osservatorio Astronomico di Palermo. Il dataset a mia disposizione comprende **10,000 curve di luce simulate** ed etichettate di RSG **suddivise in 5 classi** in base ai periodi di variazione della luminosità, come suggerito dalla letteratura. Tuttavia, le classi risultano **sbilanciate**, rispecchiando la distribuzione osservativa reale, poiché il dataset è stato progettato proprio per simulare le osservazioni nel modo più fedele possibile.

Di seguito sono elencati i nomi delle classi presenti nel dataset, accompagnati dalla rispettiva percentuale di distribuzione e dal metodo impiegato per la loro simulazione:
1. **CONSTANT** 15% (Soraisam 2020), non presentano variazione, ma solo rumore statistico (quantificato come tipico dalle osservazioni già raccolte);
2. **SPonly** 7% (Kiss 2006, Yang & Jiang 2011), sinusoidi regolari con periodo corto (+ rumore);
3. **LSPonly** 29% (Percy & Sato 2009, Yang & Jiang 2011), sinusoidi regolari con periodo lungo (+ rumore);
4. **SP+LSP** 14%, somma dei casi 2+3 (+ rumore);
5. **IRREGULAR** 35% (Yang & Jiang 2011), simulate come somma dei casi 2+3, ma con periodo e ampiezza variabili rapidamente nel tempo (+ rumore).

Le curve di luce sono state simulate su 912 giorni, pari a tre semetri osservativi e due di riposo, con valori distribuiti attorno a zero in quanto è stato sottratto il valore medio dell'intera curva. Nei semestri di riposo la mancanza di dati è stata flaggata ad un valore arbitrario pari a -99. Comunque, nei semestri osservativi vi sono anche dei valori pari a -99, in quanto è stata pure simulata una eventuale mancanza di dati nelle osservazioni (ad es. notti di maltempo, problemi strumentali,...).

Nel mio progetto affronterò un **problema di classificazione multiclasse**, cercando di classificare le curve di luce in base alle loro caratteristiche. A questo scopo, utilizzerò come **features i singoli punti** che compongono ciascuna curva di luce.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib qt

In [41]:
#serve a collegare il notebook alla cartella drive creata per permettere la visualizzazione delle immagini
# Importa le librerie necessarie
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
from PIL import Image

# Autenticazione e creazione dell'oggetto Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Crea un server locale e autentica
drive = GoogleDrive(gauth)

# Sostituisci con l'ID della tua cartella in Google Drive
folder_id = 'https://drive.google.com/drive/folders/1SrkE7oIKBcVaYZWv0x9bGLKpoYB-7aL8?usp=drive_link'  # Trova l'ID dalla URL della tua cartella

# Crea una cartella locale per scaricare le immagini
local_folder = 'progetto_Lauriano'
os.makedirs(local_folder, exist_ok=True)

# Recupera i file nella cartella di Google Drive
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and mimeType='image/jpeg' or mimeType='image/png'"}).GetList()

# Scarica tutte le immagini
for file in file_list:
    print(f'Downloading: {file["title"]}')
    downloaded_file = drive.CreateFile({'id': file['id']})
    downloaded_file.GetContentFile(os.path.join(local_folder, file['title']))

# Visualizza tutte le immagini scaricate
for filename in os.listdir(local_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(local_folder, filename)
        img = Image.open(img_path)
        plt.imshow(img)
        plt.title(filename)
        plt.axis('off')  # Disattiva gli assi
        plt.show()

InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [2]:
#importo il dataset
data = np.load('all_simulated_lc_v4.npz')

In [3]:
#mostra le chiavi disponibili nel file
#il file 'npz' permette di salvato molti array all'interno di un singolo elemento non compresso
print("Chiavi presenti nel file NPZ:", data.keys())

# itera su ciascun array e ne stampa la forma (numero di righe e colonne)
for key in data.keys():
    array = data[key]
    print(f"'{key}' ha la forma {array.shape}")

Chiavi presenti nel file NPZ: KeysView(NpzFile 'all_simulated_lc_v4.npz' with keys: sim_type, all_simulated_lc, bkg_sigma, good_fraction, seasonal_array...)
'sim_type' ha la forma (10000,)
'all_simulated_lc' ha la forma (10000, 912)
'bkg_sigma' ha la forma (10000,)
'good_fraction' ha la forma (10000,)
'seasonal_array' ha la forma (912,)
'period1' ha la forma (10000,)
'amplitude1' ha la forma (10000,)
'phase1' ha la forma (10000,)
'period2' ha la forma (10000,)
'amplitude2' ha la forma (10000,)
'phase2' ha la forma (10000,)


Sebbene il dataset presenti tante informazioni, le uniche che utilizzerò saranno le curve di luce contenute in 'all_simulated_lc'. Non farò uso di periodi, fasi, ampiezze ed errori perché sono proprio i valori utilizzati per costruire le curve! Pertanto, se dovessi classificare le curve da dati reali non sarebbero informazioni in mio possesso.

In [4]:
### PLOT N-ESIMA CURVA DI LUCE ###
all_simulated_lc = data['all_simulated_lc'] #estrae le curve di luce
x=np.arange(len(all_simulated_lc[0])) #determina la lunghezza delle curve (912 giorni)
n=0 #seleziona la n-esima curva di luce

In [20]:
# Crea il grafico della n-esima curva
plt.scatter(x, all_simulated_lc[n], c='blue', label='Osservazioni', s=4, alpha=0.7)
plt.xlabel('Tempo [giorni]')
plt.ylabel('Flusso normalizzato')
plt.title(f'SP+LSP')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(-2,2)
plt.tight_layout()
plt.savefig(f'progetto\\lc_SP+LSP_n{n}.jpg',dpi=150)

Di seguito si riporta un esempio di ciascuna classe di curva di luce, per dare un'idea dei dati:

<img src="progetto/lc_irregular_n0.jpg" alt="irregular" style="width: 300px;"/>
<img src="progetto/lc_SPonly_n2.jpg" alt="SPonly" style="width: 300px;"/>
<img src="progetto/lc_LSPonly_n3.jpg" alt="LSPonly" style="width: 300px;"/>
<img src="progetto/lc_SP+LSP_n5.jpg" alt="SP+LSP_new" style="width: 300px;"/>
<img src="progetto/lc_constant_n19.jpg" alt="constant" style="width: 300px;"/>


Nella sezione successiva si interpolano le curve di luce per i dati mancanti all'interno delle finestre osservative, di modo da avere lo stesso numero di features fra tutte le curve e nelle stesse posizioni (i periodi dei semestri sono gli stessi per tutte le curve).
Per l'interpolazione 1D sono state provate diverse strategie che saranno di seguito descritte.

In [5]:
from scipy.interpolate import interp1d
from scipy.interpolate import Akima1DInterpolator

#maschero i dati dei semestri di riposo così da lavorare soltanto sull'interpolazione 
# all'interno dei tre semetri osservativi
seasonal_mask = data['seasonal_array'] == 1 #valore booleano: 1 semestre attivo, 0 semestre di riposo
x_filt = x[seasonal_mask] #len(x_filt)=546

#ciclo su tutte le curve di luce nel dataset
masked_curves = []
interpolated_curves = []
i=0
for i in range(len(all_simulated_lc)):
    lc_simulation = all_simulated_lc[i] #estrae le curve
    lc_masked = lc_simulation[seasonal_mask] #applica la maschera
    masked_curves.append(lc_masked) #salva la i-esima curva mascherata

    ### Interpolazione punti all'interno dei semestri osservativi ###
    #all'interno dei semestri di riposo possono mancare dati per via di meteo avverso o malfunzionamenti strumentali
    #per garantire che tutti gli array abbiano lo stesso numero di features (anche presenti nei medesimi giorni)
    #si interpolano i dati
    all_indices = np.arange(len(lc_masked)) #546
    valid_indices = np.flatnonzero(lc_masked != -99) #x: posizioni in cui si registrano dati
    valid_values = lc_masked[valid_indices]          #y: valori dei dati alle posizioni 'valid_indices'

    # Funzione di interpolazione (quadratica)
    interp_func = Akima1DInterpolator(valid_indices, valid_values, extrapolate=True)
    #interp_func = interp1d(valid_indices, valid_values, kind='quadratic', bounds_error=False, fill_value="extrapolate")
    new_values = interp_func(all_indices) #applica l'interpolazione

    # Sostituzione dei valori -99 con i nuovi valori
    lc_inter = lc_masked.copy()  #nuovo array con i dati interpolati
    lc_inter[lc_inter == -99] = new_values[lc_inter == -99]
    interpolated_curves.append(lc_inter) #salva la curva interpolata

# conversione interpolated_curves in un array numpy per operazioni future
masked_curves = np.array(masked_curves)
interpolated_curves = np.array(interpolated_curves)
#np.save("interpolated_curves.npy", interpolated_curves)

In [6]:
##crea un array per le curve finali con la dimensione completa di 912 giorni
#quindi senza maschera sui semestri di riposo##
##Strategia utile ai fini di una eventuale rete RNN##
full_curves = np.copy(all_simulated_lc)

i=0
#ciclo per costruire le curve di luce finali
for i in range(len(all_simulated_lc)):
    # sostitue i valori nei giorni attivi con i valori interpolati
    full_curves[i, seasonal_mask] = interpolated_curves[i]

#salva le curve finali in un file CSV
full_curves = full_curves[:, :-2]
full_df = pd.DataFrame(full_curves)

#estraggo le classi dal dataset simulato iniziale
classes = [entry.split(',')[3].split(':')[0].strip() for entry in data['sim_type']]
full_df['target'] = classes #inserisco la colonna con le classi nel file
full_df.to_csv('interp_lc_5sem.csv', index=False) #salva un file esterno

In [7]:
#stampa delle classi presenti nel dataset (sono le stesse già anticipate sopra nel testo)
print(np.unique(classes))

['Constant' 'Irregular' 'LSPonly' 'SP+LSP' 'SPonly']


In [ ]:
#plot curva n-esima interpolata#
plt.scatter(x_filt,interpolated_curves[n], label='LC interpolata', s=2, color='#ff7f0e')    #curva interpolata
plt.scatter(x_filt,masked_curves[n], label='LC simulata', s=4, color='#0000CD', marker='o') #curva simulata

type='Irregular'
inter_type='inter1d quadratica'
plt.title(f'{type} {inter_type}')
plt.xlabel('Tempo [giorni]')
plt.ylim(-2,2)
plt.ylabel('Flusso normalizzato')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
#plt.savefig(f'progetto\\lc_{type}_n{n}_{inter_type}.jpg',dpi=150)

Descrizione delle due migliori strategie di interpolazione:

**Akima1DInterpolator** [[doc](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.Akima1DInterpolator.html)]: interpolazione cubica a tratti, evitando eccessive variazioni fra i dati utilizzati per interpolare

**inter1d** [[doc](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html)]: permette di interpolare una funzione 1d e di scegliere il grado con cui lo si vuole fare.

Di seguito si riportano i plot di tutte e cinque le classi con le due differenti tipologie di interpolazione per osservare come la akima introduca meno irregolarità nell'interpolazione e per questo la si preferisce sull'altra.

<img src="progetto/lc_irregular_n0_akima.jpg" alt="irregular_akima" style="width: 230px;"/>
<img src="progetto/lc_SPonly_n2_akima.jpg" alt="SPonly_akima" style="width: 230px;"/>
<img src="progetto/lc_LSPonly_n3_akima.jpg" alt="LSPonly_akima" style="width: 230px;"/>
<img src="progetto/lc_SP+LSP_n5_akima.jpg" alt="SP+LSP_akima" style="width: 230px;"/>
<img src="progetto/lc_constant_n19_akima.jpg" alt="constant_akima" style="width: 230px;"/>

<img src="progetto/lc_irregular_n0_inter1d quadratica.jpg" alt="irregular_inter1d" style="width: 230px;"/>
<img src="progetto/lc_SPonly_n2_inter1d quadratica.jpg" alt="SPonly_inter1d" style="width: 230px;"/>
<img src="progetto/lc_LSPonly_n3_inter1d quadratica.jpg" alt="LSPonly_inter1d" style="width: 230px;"/>
<img src="progetto/lc_SP+LSP_n5_inter1d quadratica.jpg" alt="SP+LSP_inter1d" style="width: 230px;"/>
<img src="progetto/lc_constant_n19_inter1d quadratica.jpg" alt="constant_inter1d" style="width: 230px;"/>

Come già anticipato, ogni punto della cura di luce rappresenterà una feature; infatti, sebbene ci siano sicuramente altre possibilità, come quella di fittare le curve ed estrapolare informazioni di carattere fisico come periodo, fase e ampiezza, l'approccio seguito permette di mantenere il problema di classificazioni ad un livello di astrazione maggiore, rendendo più interessante la ricerca del corretto algoritmo di intelligenza artificiale in grado di individuare pattern nei dati.

Tuttavia, dato che un alto numero di features rende computazionalmente più impegnativa l'analisi (il mio PC presenta 6 core fisici i7 9H) e con certi modelli si rischia anche di incorrere in overfitting, ho provato in un primo momento a rebinnare le curve di luce per diminuire il numero di features di un fattore 'num_points', cioè il numero di punti di cui si intendeva calcolare la media per generare un unico punto posizionato al centro dell'intervallo composto da 'num_points' punti della curva di luce.

Di seguito, commentato, riporto l'algoritmo scritto, sebbene non sia poi stato utilizzato perché si perdeva informazione sui massimi e minimi locali.

In [ ]:
'''
### REBINNING ###

from joblib import Parallel, delayed

# Assuming active_blocks is already defined and contains tuples of (start, end) for each active block
# Function to check if a point is within any active block
def is_within_active_blocks(point, active_blocks):
    return any(start <= point <= end for start, end in active_blocks)

### FUNZIONE DI REBINNING
def rebin_curve(lc_inter, x_filt, num_points):
    # Applica la maschera selezionando solo i punti interpolati e filtrando valori -99
    valid_indices = np.where(lc_inter != -99)[0]
    valid_x = x_filt[valid_indices]
    valid_y = lc_inter[valid_indices]
    
    # Array per salvare i dati rebinnati
    x_new = []
    y_new = []

    for i in np.arange(0, len(valid_x) - num_points + 1, num_points):
        # Calcola la media degli m consecutivi punti in x
        xmid = np.mean(valid_x[i:i + num_points])

        # controlla se xmid è all'interno del blocco attivo (se num_points non è un divisore della len(active_block) allora un punto cadrà nel semestre di riposo)
        if is_within_active_blocks(xmid, active_blocks):
            x_new.append(xmid)

            # Calcola la media di m consecutivi punti in y
            ymid = np.mean(valid_y[i:i + num_points])
            y_new.append(ymid)
    
    return np.array(x_new), np.array(y_new)

num_points = 2  # numero di punti rebinnati
results = Parallel(n_jobs=-1)(delayed(rebin_curve)(interpolated_curves[i], x_filt, num_points) for i in range(len(interpolated_curves)))
rebin_lc_x, rebin_lc_y = zip(*results) #separo i risultati rebinned in due array x e y

# plot
plt.scatter(rebin_lc_x[n], rebin_lc_y[n], s=2, color='#ff7f0e', label=f'Interpolati bin{num_points}')
plt.scatter(x_filt, masked_curves[n], s=3.5, color='#0000CD', label='Osservazioni')
plt.ylim(-1.2, 1.2) #serve per visualizzare correttamente le osservazioni originali
plt.xlabel('Tempo [giorni]')
plt.ylabel('Flusso normalizzato')
plt.title(f'Curva di luce {n + 1} - Rebinned con {num_points}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
'''

# Random Forest e XGBoost
Un primo approccio alla classificazione è stato compiuto impiegando i modelli *Random Forest* e *XGBoost*, di cui qui viene fornita una breve descizione.  

**Random Forest** [[doc](https://scikit-learn.org/1.5/modules/generated/sklearn.ensemble.RandomForestClassifier.html)] è un insieme di alberi decisionali, ognuno con specialità leggermente diverse, e la previsione finale è interpretabile come una decisione collettiva, presa dopo aver considerato il contributo di tutti gli alberi.

Di seguito uno schema rappresentativo:

<img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*M6ZBzDoQH1Vs3LRCpT5v2Q.png" alt="Alt text" width="500"/>



**XGBoost (eXtreme Gradient Boosting)**: il concetto fondamentale di XGBoost, come per altri metodi di boosting, è quello di aggiungere nuovi modelli all'ensemble in modo sequenziale. Tuttavia, a differenza di Random Forest, dove gli alberi vengono fatti crescere in parallelo, i metodi di boosting addestrano i modelli in serie uno dopo l'altro, con ogni nuovo albero che contribuisce a correggere gli errori commessi dall'albero precedente.

#### *Osservazioni importanti !*
*Standardizzazione/normalizzazione*: per questi due modelli non è necessario standardizzare o normalizzare le feature, in quanto gli alberi decisionali (sui quali si basa pure XGBoost) non vengono influenzati dalle scale delle variabili;

*Etichette*: Random Forest accetta categorie (quindi parole) come etichette, mentre XGBoost richiede etichette scalari. Pertanto, per quest'ultimo si utilizza la funzione *LabelEncoder* per convertirle (N.B. funzione diversa da *OrdinalEncoder*). 

In [8]:
import seaborn as sns

'''
# Funzione per applicare l'errore poissoniano su una curva di luce
# siccome le curve sono normalizzate a 0, anche un poissoniano introdurrebbe un errore troppo grande
#ad esempio un punto y=0.5 scatterato in [-poisson_noise, poisson_noise] assumerebbe valori da [-0.2, 1.2]

def poisson_noise(curve):
    poisson_noise = np.sqrt(np.abs(curve)) #calcola l'errore
    curve_with_noise = curve + np.random.uniform(-poisson_noise, poisson_noise) #applica l'errore
    return curve_with_noise
'''

### CREO TRE DATAFRAME DI PANDAS, ciascuno contenente rispettivamente le classi sbilanciate, upsampled e downsampled
#per evitare il problema del data-leakage
i=0
df = pd.DataFrame(interpolated_curves, columns=[f'feature_{i}' for i in range(np.array(interpolated_curves).shape[1])])
df_upsampling = pd.DataFrame(interpolated_curves, columns=[f'feature_{i}' for i in range(np.array(interpolated_curves).shape[1])])
df_downsampling = pd.DataFrame(interpolated_curves, columns=[f'feature_{i}' for i in range(np.array(interpolated_curves).shape[1])])
df['target'] = df_upsampling['target'] = df_downsampling['target'] = classes

#estraggo features e classi per ciascuno dei tre dataset
X = df.drop(columns=['target'])
y = df['target']

X_up = df_upsampling.drop(columns=['target'])
y_up = df_upsampling['target']
#X_up_poisson = np.apply_along_axis(poisson_noise, axis=1, arr=X_up_res) #per applicare il rumore poissoniano

X_down = df_downsampling.drop(columns=['target'])
y_down = df_downsampling['target']

In [9]:
##qui si separa il 20% dei training dataset a formare i test
from sklearn.model_selection import train_test_split

#split train e test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_up, X_test_up, y_train_up, y_test_up = train_test_split(X_up, y_up, test_size=0.2, random_state=42)
X_train_down, X_test_down, y_train_down, y_test_down = train_test_split(X_down, y_down, test_size=0.2, random_state=42)

Dal momento che le classi all'interno dei dati risultano sbilanciate, per evitare di introdurre un bias nell'apprendimento e che i modelli finiscano per favorire le classi più rappresentate si intende adesso bilanciare le classi del train dataset. Infatti, è importante che i mdelli apprendano da dati bilanciati, ma possono essere testati su un dataset sbilanciato. Quest'ultimo punto è assai importante per due motivi:

- il test non deve mai essere stato 'visto' ed esercitandovi data augmentation si rischia oltre che il data leackage anche di avere degli esempi uguali a quelli contenuti nel training se si utilizza, ad esempio, Randomoversampler;

- nel caso specifico di questo problema di classificazione, testando l'algoritmo su un test sbilanciato si tiene fede al reale sbilanciamento delle osservazioni.


Le funzioni di data augmentation messe a disposizione da scikit-learn sono le seguenti:

*Upsampling* [[random](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.RandomOverSampler.html)] [[SMOTE](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)]: è una strategia di 'data augmentation' per aumentare il numero di esempi delle classi meno rappresentate, normalizzando al valore della classe maggiore;

*Downsamplign* [[random](https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html)]: normalizza le popolazioni di tutte le classi a quella meno rappresentata, scegliendo randomicamente quali dati mantenere.

In [ ]:
### DATA AUGMENTATION ###

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

#Esistono due funzioni per fare data augmentation 'SMOTE' e 'RandomOverSampler'
# che sono state entrambe testate ed i cui risultati erano confrontabili.
#SMOTE, come altri algoritmi statistici, lavora con dati numerici e richiede sia insiemi 
#di caratteristiche che insiemi di etichette.
'''
ros = RandomOverSampler(sampling_strategy='not majority', random_state=42)
X_train_up_res, y_train_up_res = ros.fit_resample(X_train_up, y_train_up)
'''
smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_up_res, y_train_up_res = smote.fit_resample(X_train_up, y_train_up)
#X_up_poisson = np.apply_along_axis(poisson_noise, axis=1, arr=X_up_res)


## DOWNSAPLING ##
#è un'altra strategia, utile a non aggiungere rumore che potrebbe creare distorsioni nei dati
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=42)
X_train_down_res, y_train_down_res = rus.fit_resample(X_train_down, y_train_down)

## Analisi della distribuzione delle classi ##
def plot_class_distribution(y, title,filename):
    plt.figure(figsize=(6, 5))
    sns.countplot(x=y,color='orange',width=0.6)
    plt.title(title)
    plt.xlabel("Classi")
    plt.ylabel("Frequenza")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(filename, format='jpg', dpi=150)
    plt.close()  #chiude la figura per liberare memoria

#plot della distribuzione delle classi originarie e di quelle down/up-sampled
plot_class_distribution(y, "Distribuzione delle Classi - Sbilanciate", "progetto//train_unbal.jpg")
plot_class_distribution(y_train_up_res, "Distribuzione delle Classi - Bilanciata upsampling", "progetto//train_upres.jpg")
plot_class_distribution(y_train_down_res, "Distribuzione delle Classi - Bilanciata downsapling", "progetto//train_downres.jpg")

Popolazione delle classi del training dataset nel caso originale e nei casi sia di upsampling che di downsampling.

<img src="progetto/train_unbal.jpg" alt="y" style="width: 300px;"/>
<img src="progetto/train_upres.jpg" alt="y_train_up_res" style="width: 300px;"/>
<img src="progetto/train_downres.jpg" alt="y_train_down_res" style="width: 300px;"/>

### Random Forest e XGBoost

Random forest, come del resto tutti i modelli di ML, si basa su degli iperparametri, il cui valore può significativamente influenzare i risultati del modello. Poiché non si conoscono a priori i migliori valori di questi parametri e ricercarli manualmente sarebbe alquanto lungo e fortunoso, si preferisce scegliere diversi valori presumibilmente corretti ed esplorarli nelle varie combinazioni per mezzo della funzione *[GridSearchCV](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.GridSearchCV.html)*.

*Attenzione!*
Nel codice è commentata la gridia inizialmente utilizzata, ma sotto si riporta una griglia con i soli valori best osservati, di modo da evitare all'utente che compilerà una lunga attesa.

In [11]:
### RANDOM FOREST ###
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# ricerca iperparametri per Random Forest
'''
rf_param_grid = {
    'n_estimators': [50,100,200],
    'max_depth': [10,20,30],
    'min_samples_split': [2,5,7],
    'min_samples_leaf': [1,2],
    'bootstrap': [True,False]
}
'''
rf_param_grid = {
    'n_estimators': [200],
    'max_depth': [20],
    'min_samples_split': [5],
    'min_samples_leaf': [2],
    'bootstrap': [False]
}

# GridSearchCV per Random Forest (dati sbilanciati)
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1) #n_jobs=-1 indica di parallelizzare su tutti i core fisici disponibili
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid,
                               cv=3, verbose=1, n_jobs=-1)
rf_grid_search.fit(X_train, y_train)  # addestramento con dati sbilanciati
print("Migliori iperparametri Random Forest (sbilanciato):", rf_grid_search.best_params_)

# Previsione e report per Random Forest (dati sbilanciati)
rf_best_model = rf_grid_search.best_estimator_
y_pred_rf = rf_best_model.predict(X_test)
print("Random Forest (sbilanciato) - Report di classificazione:")
print(classification_report(y_test, y_pred_rf))

#---------------#

# GridSearchCV per Random Forest (bilanciato upsampling)
rf_grid_search_up = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid,
                                     cv=3, verbose=1, n_jobs=-1)
rf_grid_search_up.fit(X_train_up_res, y_train_up_res)
print("Migliori iperparametri Random Forest (bilanciato upsampling):", rf_grid_search_up.best_params_)

# Previsione e report per Random Forest (bilanciato upsampling)
rf_best_model_up = rf_grid_search_up.best_estimator_
y_pred_rf_up = rf_best_model_up.predict(X_test_up)
print("Random Forest (bilanciato upsampling) - Report di classificazione:")
print(classification_report(y_test_up, y_pred_rf_up))

#---------------#

# GridSearchCV per Random Forest (bilanciato downsampling)
rf_grid_search_down = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid,
                                     cv=3, verbose=1, n_jobs=-1)
rf_grid_search_down.fit(X_train_down_res, y_train_down_res)
print("Migliori iperparametri Random Forest (bilanciato downsampling):", rf_grid_search_down.best_params_)

# Previsione e report per Random Forest (bilanciati downsampling)
rf_best_model_down = rf_grid_search_down.best_estimator_
y_pred_rf_down = rf_best_model_down.predict(X_test_down)
print("Random Forest (bilanciato downsampling) - Report di classificazione:")
print(classification_report(y_test_down, y_pred_rf_down))

# accuratezza modelli ottimizzati
print("Random Forest (sbilanciato) - Accuratezza:", accuracy_score(y_test, y_pred_rf))
print("Random Forest bilanciato upsampling - Accuratezza:", accuracy_score(y_test_up, y_pred_rf_up))
print("Random Forest bilanciato downsampling - Accuratezza:", accuracy_score(y_test_down, y_pred_rf_down))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Migliori iperparametri Random Forest (sbilanciato): {'bootstrap': False, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Random Forest (sbilanciato) - Report di classificazione:
              precision    recall  f1-score   support

    Constant       0.77      0.96      0.85       271
   Irregular       0.95      1.00      0.98       716
     LSPonly       0.87      0.88      0.88       592
      SP+LSP       0.67      0.63      0.65       274
      SPonly       0.47      0.19      0.27       147

    accuracy                           0.85      2000
   macro avg       0.75      0.73      0.72      2000
weighted avg       0.83      0.85      0.83      2000

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Migliori iperparametri Random Forest (bilanciato upsampling): {'bootstrap': False, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Random For

A parte i punteggi dettagliati che si osservano dopo la compilazione della cella superiore, si preferisce generare una matrice di confusione per osservare effettivamente quanti dei dati risultano correttamente predetti. Di tale matrice si definisce qui sotto una funzione che verrà richiamata ad ogni modello e per la quale l'utente avrà la possibilità di modificare titolo del plot e nome del file ad ogni nuova compilazione. 

In [12]:
## DEFINIZIONE FUNZIONE MATRICE DI CONFUSIONE ##
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

title='Matrici di confusione Random Forest con interpolazione akima e upsampling SMOTE'
filename='RF_akima_smote'

def conf_matrix(cm1, cm2, cm3, title1, title2, title3, class_labels):
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    fig.suptitle(f'{title}', fontsize=16, y=0.97)  # Aggiunge titolo generale

    # prima matrice di confusione (dati sbilanciati)
    sns.heatmap(cm1, annot=True, fmt='d', cmap='Blues', ax=axes[0],
                xticklabels=class_labels, yticklabels=class_labels)
    axes[0].set_title(title1)
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')
    axes[0].tick_params(axis='x', rotation=45)

    # seconda matrice (dati upsampled)
    sns.heatmap(cm2, annot=True, fmt='d', cmap='Blues', ax=axes[1],
                xticklabels=class_labels, yticklabels=class_labels)
    axes[1].set_title(title2)
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('Actual')
    axes[1].tick_params(axis='x', rotation=45)

    # terza matrice (dati downsampled)
    sns.heatmap(cm3, annot=True, fmt='d', cmap='Blues', ax=axes[2],
                xticklabels=class_labels, yticklabels=class_labels)
    axes[2].set_title(title3)
    axes[2].set_xlabel('Predicted')
    axes[2].set_ylabel('Actual')
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(f'progetto//cm_{filename}.jpg', dpi=150, bbox_inches='tight')
    plt.show()

class_labels = sorted(y.unique()) #etichette delle classi

##plot matrici di confusione Random Forest
# reazione delle matrici
cm_rf = confusion_matrix(y_test, y_pred_rf)
cm_rf_up = confusion_matrix(y_test_up, y_pred_rf_up)
cm_rf_down = confusion_matrix(y_test_down, y_pred_rf_down)

#plot
conf_matrix(cm_rf, cm_rf_up, cm_rf_down,
                "train sbilanciato", "train upsampled", "train downsampled", class_labels)

Adesso si definisce una matrice per la stampa delle *curve ROC (Receiver Operating Characteristic)*, cioè un grafico che mostra le prestazioni di un modello di classificazione per diversi valori di soglia di decisione. La curva rappresenta la relazione tra tasso di veri positivi (TPR) e tasso di falsi positivi (FPR), la proporzione di campioni negativi classificati erroneamente come positivi. Di ciascuna curva si calcola anche
l'*AUC (Area Under the Curve)* che riassume le prestazioni del modello; infatti, AUC pari ad uno indica un modello perfetto, che distingue correttamente tutte le classi.

In [13]:
## FUNZIONE GRAFICO CURVA ROC E CALCOLO AUC ##
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

filename='curve_ROC_RF_akima'
title_plot= 'Confronto Curve ROC Medie - Random Forest'

#funzione per calcolare e tracciare le curve ROC medie
def plot_average_roc_curve(y_tests, y_pred_probas, titles, class_labels):
    n_classes = len(class_labels)  # numero di classi nel dataset
    plt.figure(figsize=(10, 8))  # crea una figura per plottare le curve ROC

    ##dati per la curva ROC media: prende in considerazione l'accuratezza media di tutte le classi##
    mean_fpr = np.linspace(0, 1, 100)  # creazione di un array per FPR
    
    for y_test, y_pred_proba, title in zip(y_tests, y_pred_probas, titles):
        #binarizza le etichette per ROC multi-classe
        y_test_bin = label_binarize(y_test, classes=class_labels)  # conversione etichette in formato binario
        
        tprs = []  # lista per memorizzare TPR
        aucs = []  # lista per memorizzare le AUC
        
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])  # calcola i valori FPR e TPR per la classe i
            roc_auc = auc(fpr, tpr)  # calcola l'AUC per la curva ROC della classe i
            
            #interpolazione per ottenere una curva liscia
            tpr_interp = np.interp(mean_fpr, fpr, tpr)  # interpolazione dei TPR
            tpr_interp[0] = 0.0
            tprs.append(tpr_interp)  # aggiunge la curva TPR interpolata alla lista
            aucs.append(roc_auc)     # aggiunge AUC alla lista
        
        # media delle TPR e AUC
        mean_tpr = np.mean(tprs, axis=0)   #calcola la media delle TPR per tutte le classi
        mean_auc = auc(mean_fpr, mean_tpr) #calcola l'AUC media

        # curva ROC media
        plt.plot(mean_fpr, mean_tpr, label=f"{title} (AUC = {mean_auc:.4f})")  # disegna la curva ROC media con la legenda

    plt.plot([0, 1], [0, 1], "k--")  # disegna una linea diagonale di riferimento (random chance)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f'{title_plot}')
    plt.legend(loc="lower right")
    plt.savefig(f'progetto//{filename}.jpg', dpi=150)
    plt.show()

class_labels = sorted(y.unique())  # etichette delle classi

#probabilità predette
y_pred_proba_rf = rf_best_model.predict_proba(X_test)  # probabilità previste per il modello Random Forest con dati sbilanciati
y_pred_proba_rf_up = rf_best_model_up.predict_proba(X_test_up)  # dati upsampled
y_pred_proba_rf_down = rf_best_model_down.predict_proba(X_test_down)  # dati downsampled

# plot la curva ROC media
plot_average_roc_curve(
    y_tests=[y_test, y_test_up, y_test_down],  # dati di test per ciascun modello
    y_pred_probas=[y_pred_proba_rf, y_pred_proba_rf_up, y_pred_proba_rf_down],  #probabilità previste per ciascun modello
    titles=["Dati Sbilanciati", "Dati Upsampled", "Dati Downsampled"], 
    class_labels=class_labels)

In [14]:
### XGBoost ###
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

label_encoder = LabelEncoder() # codifica delle etichette scalari per XGBoost

#sbilanciati
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

#bilanciati upsampling
y_train_up_enc = label_encoder.fit_transform(y_train_up_res)
y_test_up_enc = label_encoder.fit_transform(y_test_up)

#bilanciati downsampling
y_train_down_enc = label_encoder.transform(y_train_down_res)
y_test_down_enc = label_encoder.transform(y_test_down)

# ricerca iperparametri per XGBoost
'''
xgb_param_grid = {
    'n_estimators': [50,100,200],
    'max_depth': [3,6,10],
    'learning_rate': [0.01,0.1,0.2],
    'subsample': [0.8,1.0],
    'colsample_bytree': [0.8,1.0]
}
'''
xgb_param_grid = {
    'n_estimators': [200],
    'max_depth': [20],
    'learning_rate': [0.2],
    'subsample': [1.0],
    'colsample_bytree': [0.8]
}

# GridSearchCV (dati sbilanciati)
xgb_model = XGBClassifier(eval_metric='mlogloss', n_jobs=-1)
# mlogloss: calcola la perdita logaritmica per problemi di classificazione multi-classe, 
# misurando quanto bene il modello predice le probabilità delle classi corrette
xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid,
                                cv=3, verbose=1, n_jobs=-1)
xgb_grid_search.fit(X_train, y_train_enc)  # addestramento con dati sbilanciati
print("Migliori iperparametri XGBoost (sbilanciato):", xgb_grid_search.best_params_)

# Previsione e report (dati sbilanciati)
xgb_best_model = xgb_grid_search.best_estimator_
y_pred_xgb = xgb_best_model.predict(X_test)
y_pred_xgb = label_encoder.inverse_transform(y_pred_xgb)  # decodifica delle etichette
print("XGBoost (sbilanciato) - Report di classificazione:")
print(classification_report(y_test, y_pred_xgb))

#---------------#

# GridSearchCV (upsampling)
xgb_grid_search_up = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid,
                                      cv=3, verbose=1, n_jobs=-1)
xgb_grid_search_up.fit(X_train_up_res, y_train_up_enc) #dati aumentati
print("Migliori iperparametri XGBoost (upsampling):", xgb_grid_search_up.best_params_)

# Previsione e report (upsampling)
xgb_best_model_up = xgb_grid_search_up.best_estimator_
y_pred_xgb_up = xgb_best_model_up.predict(X_test_up)
y_pred_xgb_up = label_encoder.inverse_transform(y_pred_xgb_up)
print("XGBoost bilanciato upsampling - Report di classificazione:")
print(classification_report(y_test_up, y_pred_xgb_up))

#---------------#

# GridSearchCV (downsampling)
xgb_grid_search_down = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid,
                                      cv=3, verbose=1, n_jobs=-1)
xgb_grid_search_down.fit(X_train_down_res, y_train_down_enc)
print("Migliori iperparametri XGBoost (downsapling):", xgb_grid_search_down.best_params_)

# previsione e report (downsapling)
xgb_best_model_down = xgb_grid_search_down.best_estimator_
y_pred_xgb_down = xgb_best_model_down.predict(X_test_down)
y_pred_xgb_down = label_encoder.inverse_transform(y_pred_xgb_down)
print("XGBoost bilanciato downsampling - Report di classificazione:")
print(classification_report(y_test_down, y_pred_xgb_down))

#accuratezza modelli ottimizzati
print("XGBoost (sbilanciato) - Accuratezza:", accuracy_score(y_test, y_pred_xgb))
print("XGBoost bilanciato upsampling SMOTE - Accuratezza:", accuracy_score(y_test_up, y_pred_xgb_up))
print("XGBoost bilanciato downsampling - Accuratezza:", accuracy_score(y_test_down, y_pred_xgb_down))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Migliori iperparametri XGBoost (sbilanciato): {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 20, 'n_estimators': 200, 'subsample': 1.0}
XGBoost (sbilanciato) - Report di classificazione:
              precision    recall  f1-score   support

    Constant       0.81      0.94      0.87       271
   Irregular       0.95      1.00      0.98       716
     LSPonly       0.89      0.91      0.90       592
      SP+LSP       0.65      0.60      0.62       274
      SPonly       0.42      0.21      0.28       147

    accuracy                           0.85      2000
   macro avg       0.74      0.73      0.73      2000
weighted avg       0.83      0.85      0.84      2000

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Migliori iperparametri XGBoost (upsampling): {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 20, 'n_estimators': 200, 'subsample': 1.0}
XGBoost bilanciato upsampling - Report di c

In [ ]:
#richiamo la funzione e stampo la matrice di confusione XGBoost

title='Matrici di confusione XGBoost con interpolazione akima e upsampling SMOTE'
filename='XGB_akima_smote'

cm_xgb = confusion_matrix(y_test, y_pred_xgb)
cm_xgb_up = confusion_matrix(y_test_up, y_pred_xgb_up)
cm_xgb_down = confusion_matrix(y_test_down, y_pred_xgb_down)

conf_matrix(cm_xgb, cm_xgb_up, cm_xgb_down,
        "train sbilanciato", "train upsampled", "train downsampled", class_labels)

Di seguito sono riportate le matrici di confusione di *Random Forest* e *XGBoost*. Si osserva che i risultati sono confrontabili tra loro; inoltre, è ragionevole desumere la presenza di overfitting sulla classe 'irregular' che oltre ad essere quella più rappresentata nel dataset di partenza, manifesta la maggiore etereogeneità. È anche evidente come le classi più confuse sono *SP+LSP* e *SPonly*, risultato attenbile alla luce del fatto che entrambe presentano un pattern molto evidente nel breve periodo, con la differenza che la prima classe sarebbe caratterizzata pure da una periodicità maggiore (LSP) che però, evidentemente, non viene colta da questi algoritmi.
Si osserva anche una leggera confusione tra le classi *constant* e *LSPonly*: pure questa prevedibile in quanto il pattern sul lungo periodo se non colto può essere ben confuso con un pattern costante.

<img src="progetto/cm_RF_akima_smote.jpg" alt="y" style="width: 1000px;"/>
<img src="progetto/cm_XGB_akima_smote.jpg" alt="y" style="width: 1000px;"/>

In [14]:
## richiamo la funzione e stampo le curve ROC per XGBoost ##

filename='curve_ROC_XGB_akima'
title_plot= 'Confronto Curve ROC Medie - XGBoost'

y_pred_proba_xgb = xgb_best_model.predict_proba(X_test)  # probabilità previste per il modello Random Forest con dati sbilanciati
y_pred_proba_xgb_up = xgb_best_model_up.predict_proba(X_test_up)  # dati upsampled
y_pred_proba_xgb_down = xgb_best_model_down.predict_proba(X_test_down)  # dati downsampled

# plot la curva ROC media
plot_average_roc_curve(
    y_tests=[y_test, y_test_up, y_test_down],  # dati di test per ciascun modello
    y_pred_probas=[y_pred_proba_xgb, y_pred_proba_xgb_up, y_pred_proba_xgb_down],  #probabilità previste per ciascun modello
    titles=["Dati Sbilanciati", "Dati Upsampled", "Dati Downsampled"], 
    class_labels=class_labels)

<img src="progetto/curve_ROC_RF_akima.jpg" alt="y" style="width: 500px;"/>
<img src="progetto/curve_ROC_XGB_akima.jpg" alt="y" style="width: 500px;"/>

## RETE NEURALE

Per ricercare un algoritmo più performante, si preferisce adesse esplorare le reti neurali e come primo approccio si crea una rete tradizionale (ANN), quindi composta soltanto da layers densi. Nel caso in cui si analizza il dataset 'interpolated_curves' con i dati filtrati rispetto ai semestri di riposo, il numero di features è 546, pertanto impiego 4 layers densi andando riducendo progressivamente la dimensione di ciascuno, fino ad un output di 5, equivalente al numero di classi da classificare.

In [15]:
#librerie reti neurali
#si importano tutte prima nel caso l'utente volesse compilare una soltanto delle rete di seguito presentate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras import Sequential, regularizers, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow as tf

In [ ]:
#sempre per evitare il data-leakage si crea un nuovo df per la ANN
classes = [entry.split(',')[3].split(':')[0].strip() for entry in data['sim_type']]
df_ANN_up = pd.DataFrame(interpolated_curves, columns=[f'feature_{i}' for i in range(np.array(interpolated_curves).shape[1])])
df_ANN_up['target'] = classes

#estrazione delle feature e del target
X_up = df_ANN_up.drop(columns=['target']).values
y_up = df_ANN_up['target'].values

#standardizzazione feature
scaler = StandardScaler()
X_up_scal = scaler.fit_transform(X_up)

#encoding delle etichette
y_up_enc = pd.get_dummies(y_up, dtype=int).values

# divisione dei dati in training e test set
X_train, X_test, y_train, y_test = train_test_split(X_up_scal, y_up_enc, test_size=0.2, random_state=42)

# divisione del training set in un set di addestramento e un set di validazione
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#data augmentation sul set di addestramento
smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# costruzione di una rete neurale tradizionale senza tecniche anti-overfitting
model_ann1 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_res.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # Output con 5 classi
])

# compilazione del modello
model_ann1.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# addestramento del modello
# utilizza il set di validazione per monitorare l'accuratezza del modello durante l'addestramento
history_ann1 = model_ann1.fit(
    x=X_train_res, y=y_train_res,
    validation_data=(X_val, y_val),  # set di validazione esplicito
    epochs=50,
    verbose=2)

# valutazione sul test set
test_loss, test_acc = model_ann1.evaluate(X_test, y_test, verbose=2)
print(f'\nTest Accuracy: {test_acc:.4f}')

#previsioni sul test set
y_pred_ann1 = np.argmax(model_ann1.predict(X_test), axis=-1)

Epoch 1/50


c:\Users\Mario Lauriano\miniconda3\envs\machine_learning\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


350/350 - 3s - 8ms/step - accuracy: 0.5707 - loss: 1.0679 - val_accuracy: 0.7975 - val_loss: 0.7823
Epoch 2/50
350/350 - 1s - 3ms/step - accuracy: 0.7963 - loss: 0.7293 - val_accuracy: 0.8163 - val_loss: 0.5664
Epoch 3/50
350/350 - 1s - 3ms/step - accuracy: 0.8212 - loss: 0.5288 - val_accuracy: 0.8425 - val_loss: 0.4780
Epoch 4/50
350/350 - 1s - 3ms/step - accuracy: 0.8532 - loss: 0.4266 - val_accuracy: 0.8381 - val_loss: 0.4692
Epoch 5/50
350/350 - 1s - 3ms/step - accuracy: 0.8679 - loss: 0.3698 - val_accuracy: 0.8619 - val_loss: 0.4013
Epoch 6/50
350/350 - 1s - 3ms/step - accuracy: 0.8867 - loss: 0.3236 - val_accuracy: 0.8606 - val_loss: 0.4098
Epoch 7/50
350/350 - 1s - 3ms/step - accuracy: 0.8976 - loss: 0.2906 - val_accuracy: 0.8656 - val_loss: 0.3891
Epoch 8/50
350/350 - 1s - 3ms/step - accuracy: 0.9112 - loss: 0.2609 - val_accuracy: 0.8687 - val_loss: 0.4014
Epoch 9/50
350/350 - 1s - 3ms/step - accuracy: 0.9173 - loss: 0.2404 - val_accuracy: 0.8644 - val_loss: 0.3979
Epoch 10/50


In [27]:
### Funzione per valutare le prestazioni di una rete neurale ###

from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import plot_model

def evaluate_model_performance(model, X_test, y_test, y_pred, history, model_name='neural_network'):
    """
    Genera la matrice di confusione, lo schema della rete e il grafico della loss durante l'addestramento.

    Parametri
        model (tf.keras.Model): modello di rete neurale addestrato;
        X_test (np.array): dati di test;
        y_test (np.array): etichette di test in one-hot encoding;
        y_pred (np.array): etichette predette nel formato scalare;
        history (tf.keras.callbacks.History): oggetto della cronologia di addestramento;
        model_name (str): Nome per salvare i file di output (default: 'neural_network').
    """
    # conversione delle etichette di test in formato scalare, necessario per la matrice di confusione
    y_test_scalar = np.argmax(y_test, axis=1)

    # generazione della matrice di confusione
    conf_mat = confusion_matrix(y_test_scalar, y_pred)
    print(f'Matrice di Confusione {model_name}:')
    print(conf_mat)

    ## plot dello schema della rete neurale (la figura inserita prima) ##
    plot_model(model, show_shapes=True, to_file=f'progetto\\{model_name}_structure.png')
    
    # plot dell'andamento della curva loss durante l'addestramento
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoche')
    plt.ylabel('Loss')
    plt.title(f'Curva di apprendimento - {model_name}')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'progetto\\{model_name}_training_loss.png', dpi=72)
    plt.show()

In [30]:
# valutazione del modello e generazione dei grafici e della matrice di confusione
evaluate_model_performance(model=model_ann1, X_test=X_test, y_test=y_test, y_pred=y_pred_ann1, history=history_ann1, model_name='rete_tradizionale_masked')

Matrice di Confusione rete_tradizionale_masked:
[[251   0  20   0   0]
 [  1 710   0   0   5]
 [ 32   1 547  10   2]
 [  2  13  28 168  63]
 [  2   6   7  60  72]]


Dalla curva di apprendimento si osserva come nonostante la rete impari bene dai dati di training non riesca poi a generalizzare ai dati del set di validation, perché overfitta sui dati di training. Vedremo adesso come risolvere tale problema.

<img src="progetto/rete tradizionale_masked_training_loss.png" alt="y" style="width: 500px;"/>

Prima di proseguire, si ripropone la stessa rete con il dataset interpolato, ma non filtrato sui semestri di riposo, in prospettiva di utilizzare una RNN. Tale test vorrà appurare come i valori flaggati dei semestri di riposo non disturbino (se non per tempo computazionale) la rete, mostrando dati confrontabili con la prima.

In [ ]:
## richiamo la funzione e stampo le curve ROC per XGBoost ##

filename='curve_ROC_XGB_akima'
title_plot= 'Confronto Curve ROC Medie - XGBoost'

y_pred_proba_ann1 = xgb_best_model.predict_proba(X_test)

# plot la curva ROC media
plot_average_roc_curve(
    y_tests=[y_test, y_test_up, y_test_down],  # dati di test per ciascun modello
    y_pred_probas=[y_pred_proba_xgb, y_pred_proba_xgb_up, y_pred_proba_xgb_down],  #probabilità previste per ciascun modello
    titles=["Dati Sbilanciati", "Dati Upsampled", "Dati Downsampled"], 
    class_labels=class_labels)

In [31]:
## rete neurale tradizionale senza maschera stagionale, ma utilizzando i valori -99 nei buchi dovuti ai semestri di riposo ##
full_df_ANN = pd.read_csv('interp_lc_5sem.csv')
X = full_df_ANN.drop(columns=['target']).values
y = full_df_ANN['target'].values

scaler = StandardScaler()
X_scal = scaler.fit_transform(X)

y_enc = pd.get_dummies(y, dtype=int).values

X_train, X_test, y_train, y_test = train_test_split(X_scal, y_enc, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

model_ann2 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_res.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])

model_ann2.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history_ann2 = model_ann2.fit(
    x=X_train_res, y=y_train_res,
    validation_data=(X_val, y_val),
    epochs=50,
    verbose=2)

test_loss, test_acc = model_ann2.evaluate(X_test, y_test, verbose=2)
print(f'\nTest Accuracy: {test_acc:.4f}')
y_pred_ann2 = np.argmax(model_ann2.predict(X_test), axis=-1)

Epoch 1/50


c:\Users\Mario Lauriano\miniconda3\envs\machine_learning\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


350/350 - 2s - 7ms/step - accuracy: 0.5985 - loss: 1.0421 - val_accuracy: 0.8144 - val_loss: 0.7323
Epoch 2/50
350/350 - 1s - 3ms/step - accuracy: 0.8016 - loss: 0.6978 - val_accuracy: 0.8313 - val_loss: 0.5270
Epoch 3/50
350/350 - 1s - 3ms/step - accuracy: 0.8241 - loss: 0.5047 - val_accuracy: 0.8494 - val_loss: 0.4482
Epoch 4/50
350/350 - 1s - 3ms/step - accuracy: 0.8487 - loss: 0.4175 - val_accuracy: 0.8650 - val_loss: 0.3924
Epoch 5/50
350/350 - 1s - 3ms/step - accuracy: 0.8724 - loss: 0.3614 - val_accuracy: 0.8619 - val_loss: 0.3868
Epoch 6/50
350/350 - 1s - 3ms/step - accuracy: 0.8857 - loss: 0.3208 - val_accuracy: 0.8544 - val_loss: 0.3829
Epoch 7/50
350/350 - 1s - 3ms/step - accuracy: 0.8972 - loss: 0.2898 - val_accuracy: 0.8750 - val_loss: 0.3453
Epoch 8/50
350/350 - 1s - 3ms/step - accuracy: 0.9055 - loss: 0.2717 - val_accuracy: 0.8694 - val_loss: 0.3487
Epoch 9/50
350/350 - 1s - 3ms/step - accuracy: 0.9137 - loss: 0.2421 - val_accuracy: 0.8706 - val_loss: 0.3556
Epoch 10/50


In [32]:
evaluate_model_performance(model=model_ann2, X_test=X_test, y_test=y_test, y_pred=y_pred_ann2, history=history_ann2, model_name='rete tradizionale full_lc')

Matrice di Confusione rete tradizionale full_lc:
[[242   0  28   0   1]
 [  1 707   0   5   3]
 [ 30   1 553   7   1]
 [  2  12  31 175  54]
 [  4   4   3  66  70]]


La matrice di confusione con questo dataset è pienamente confrontabile con la prima; inoltre, l'accuratezza media è addirittura migliore. Pertanto si sceglie di proseguire con tale dataset.

## Overfitting 

L'overfitting è causato dal fatto che un modello tende a "memorizzare" specifiche caratteristiche dei dati di training, compresi rumore e anomalie. Questo comportamento porta a un aumento delle prestazioni apparenti sui dati di validazione, soprattutto se il set di validazione è simile al set di addestramento (cosa plausibile in questo caso dato il data augmentation), ma il modello sarà meno capace di generalizzare.

È possibile adottare delle strategie per contrastarlo, ad esempio funzioni di regolazione:

*Regolarizzazione L2*: penalizza i pesi troppo grandi nel modello aggiungendo una "penalità" alla funzione di perdita. Questa penalità è proporzionale al quadrato dei valori dei pesi (da cui il nome L2). In pratica, L2 riduce la complessità del modello;

*Dropout*: disattiva casualmente una percentuale di neuroni in ogni strato durante l'allenamento, rendendo il modello meno dipendente da particolari neuroni. Questo "spegnimento" aiuta a evitare che il modello memorizzi i dati di allenamento (overfitting), poiché ogni iterazione costringe la rete a utilizzare percorsi diversi per apprendere;

*ReduceLROnPlateau*: riduce il tasso di apprendimento se la *val_loss* smette di migliorare per un certo numero di epoche consecutive. Infatti, un learning rate troppo alto può causare oscillazioni o impedire il raggiungimento del minimo locale della funzione di perdita, portando a overfitting o a un apprendimento instabile. Riducendo progressivamente il learning rate, il modello può adattarsi in modo più fine alla superficie della funzione di perdita, migliorando la generalizzazione;

*ModelCheckpoint*: salva il modello solo quando ottiene il miglior risultato sulla *val_loss*;

*Early Stopping*: l’allenamento si interrompe se la *val_loss* non migliora per 10 epoche consecutive.

In [33]:
## RETE ANTI-OVERFITTING ##

full_df_ANN = pd.read_csv('interp_lc_5sem.csv')
X_up = full_df_ANN.drop(columns=['target']).values
y_up = full_df_ANN['target'].values

scaler = StandardScaler()
X_up_scal = scaler.fit_transform(X_up)

y_up_enc = pd.get_dummies(y_up, dtype=int).values

X_train, X_test, y_train, y_test = train_test_split(X_up_scal, y_up_enc, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

model_ann3 = Sequential([
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(X_train_res.shape[1],)),
    Dropout(0.2),  # Dropout del 50% dei neuroni
    
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    
    Dense(5, activation='softmax')
])

model_ann3.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model_cnn_lstm.keras', monitor='val_loss', save_best_only=True)

history_ann3 = model_ann3.fit(
    x=X_train_res, y=y_train_res,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, lr_scheduler, checkpoint],
    verbose=2)

test_loss, test_acc = model_ann3.evaluate(X_test, y_test, verbose=2)
print(f'\nTest Accuracy: {test_acc:.4f}')
y_pred_ann3 = np.argmax(model_ann3.predict(X_test), axis=-1)

Epoch 1/50


c:\Users\Mario Lauriano\miniconda3\envs\machine_learning\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


350/350 - 3s - 8ms/step - accuracy: 0.4305 - loss: 1.8499 - val_accuracy: 0.6994 - val_loss: 1.4082 - learning_rate: 1.0000e-04
Epoch 2/50
350/350 - 1s - 4ms/step - accuracy: 0.6289 - loss: 1.5333 - val_accuracy: 0.8175 - val_loss: 1.1934 - learning_rate: 1.0000e-04
Epoch 3/50
350/350 - 1s - 4ms/step - accuracy: 0.7124 - loss: 1.3485 - val_accuracy: 0.8256 - val_loss: 1.0477 - learning_rate: 1.0000e-04
Epoch 4/50
350/350 - 1s - 4ms/step - accuracy: 0.7381 - loss: 1.2042 - val_accuracy: 0.8313 - val_loss: 0.9505 - learning_rate: 1.0000e-04
Epoch 5/50
350/350 - 1s - 4ms/step - accuracy: 0.7481 - loss: 1.1197 - val_accuracy: 0.8344 - val_loss: 0.9049 - learning_rate: 1.0000e-04
Epoch 6/50
350/350 - 1s - 4ms/step - accuracy: 0.7614 - loss: 1.0480 - val_accuracy: 0.8313 - val_loss: 0.8717 - learning_rate: 1.0000e-04
Epoch 7/50
350/350 - 1s - 4ms/step - accuracy: 0.7743 - loss: 0.9995 - val_accuracy: 0.8450 - val_loss: 0.8547 - learning_rate: 1.0000e-04
Epoch 8/50
350/350 - 1s - 4ms/step - a

In [34]:
evaluate_model_performance(model=model_ann3, X_test=X_test, y_test=y_test, y_pred=y_pred_ann3, history=history_ann3, model_name='rete tradizionale_regul_overfitting')

Matrice di Confusione rete tradizionale_regul_overfitting:
[[249   0  20   0   2]
 [  0 712   0   2   2]
 [ 40   1 545   6   0]
 [  2  12  26 152  82]
 [  3   8   2  44  90]]


Con le regolazioni aggiunte si osserva una migliore capacità di generalizzazione ai dati di validazione. 

<img src="progetto/rete tradizionale_regul_overfitting_training_loss.png" alt="y" style="width: 500px;"/>

È comunque evidente che ancora c'è da migliorare per rendere la rete capace di generalizzare meglio: un modo per farlo è semplificare il modello riducendo il numero di strati e/o diminuendo il numero di neuroni di ciascuno.

Inoltre, poiché le curve di luce sono ordinate temporalmente, si preferisce di impiegare layers convoluzionali, utili a trovare pattern nel breve periodo all'interno dei dati.

## Rete Neurale Convoluzionale (CNN)

Le reti neurali convoluzionali si distinguono da quelle tradizionali per le loro prestazioni superiori con immagini, input vocali e segnali unidimensionali (quale è una curva di luce). Sono principalmente suddivise in tre tipi di livelli, ovvero:

- convoluzionali, dove è presente un rilevatore di funzioni, definito anche kernel o filtro, che 'scorrerà' sull'array, verificando la presenza di pattern nel breve periodo (questo processo è noto come convoluzione);
- di pooling, una sorta di filtro che seleziona il punto con il valore massimo da inviare all'array di output;
- completamente connesso (FC, Fully-connected), ne serve almneo uno perché permette di connettere l'output direttamente a un nodo nel livello precedente.

*Attenzione!* I  primi layers convoluzionali hanno meno neuroni di quelli successivi, come a formare una 'gerarchia': i primi osserveranno le caratteristiche più superficiali del modello e quelli successivi andranno più nel dettaglio.

In [37]:
## CNN  ##

full_df_ANN = pd.read_csv('interp_lc_5sem.csv')
X = full_df_ANN.drop(columns=['target']).values
y = full_df_ANN['target'].values

scaler = StandardScaler()
X_scal = scaler.fit_transform(X)

y_enc = pd.get_dummies(y, dtype=int)

X_train, X_test, y_train, y_test = train_test_split(X_scal, y_enc, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Conversione da one-hot a etichette numeriche per SMOTE
y_train_lab = np.argmax(y_train.values, axis=1)

smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train_lab)

# Conversione delle etichette riequilibrate in one-hot encoding
y_train_res = pd.get_dummies(y_train_res)

# Reshape dei dati per Conv1D: (campioni, caratteristiche, 1)
X_train_res = X_train_res.reshape((X_train_res.shape[0], X_train_res.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definizione del modello CNN
model_cnn1 = Sequential([
    Conv1D(filters=32,kernel_size=7, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),

    Conv1D(filters=64, kernel_size=7, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),

    Flatten(),

    # Strati densi aggiuntivi
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.2),

    Dense(5, activation='softmax')
])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
checkpoint = ModelCheckpoint('best_model_cnn_lstm.keras', monitor='val_loss', save_best_only=True)

model_cnn1.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history_cnn1 = model_cnn1.fit(
    x=X_train_res, y=y_train_res,  # Usa X_train_res e y_train_res bilanciati
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[early_stopping, lr_scheduler, checkpoint],
    verbose=2)

test_loss, test_acc = model_cnn1.evaluate(X_test, y_test, verbose=2)
print(f'\nTest Accuracy: {test_acc:.4f}')

y_pred_cnn1 = np.argmax(model_cnn1.predict(X_test), axis=-1)

Epoch 1/50
350/350 - 8s - 23ms/step - accuracy: 0.5435 - loss: 1.2990 - val_accuracy: 0.7506 - val_loss: 0.8514 - learning_rate: 0.0010
Epoch 2/50
350/350 - 6s - 18ms/step - accuracy: 0.6414 - loss: 0.9856 - val_accuracy: 0.8169 - val_loss: 0.7754 - learning_rate: 0.0010
Epoch 3/50
350/350 - 6s - 18ms/step - accuracy: 0.6572 - loss: 0.9428 - val_accuracy: 0.7875 - val_loss: 0.7320 - learning_rate: 0.0010
Epoch 4/50
350/350 - 7s - 20ms/step - accuracy: 0.6680 - loss: 0.9134 - val_accuracy: 0.8119 - val_loss: 0.6838 - learning_rate: 0.0010
Epoch 5/50
350/350 - 7s - 19ms/step - accuracy: 0.6777 - loss: 0.8901 - val_accuracy: 0.8163 - val_loss: 0.6402 - learning_rate: 0.0010
Epoch 6/50
350/350 - 7s - 20ms/step - accuracy: 0.6819 - loss: 0.8740 - val_accuracy: 0.8269 - val_loss: 0.6351 - learning_rate: 0.0010
Epoch 7/50
350/350 - 7s - 20ms/step - accuracy: 0.6826 - loss: 0.8676 - val_accuracy: 0.8225 - val_loss: 0.6419 - learning_rate: 0.0010
Epoch 8/50
350/350 - 7s - 19ms/step - accuracy: 

In [36]:
evaluate_model_performance(model_cnn1, X_test=X_test, y_test=y_test, y_pred=y_pred_cnn1, history=history_cnn1, model_name='CNN')

Matrice di Confusione CNN:
[[247   0  22   0   2]
 [  0 709   0   4   3]
 [ 42   0 545   2   3]
 [  1  18  36 128  91]
 [  3  11   4  54  75]]


## Rete Neurale Ricorrente (RNN)

È una rete neurale profonda addestrata su dati ordinati temporalmente. Questa è la rete per cui si adattano le curve interpolate e non filtrate rispetto ai semestri osservati, perché così si riesce a conservare l'informazione temporale!

Tale rete viene suggerita in letteratura per risolvere questo genere di problemi di classificazione multiclasse [[ref](https://academic.oup.com/mnras/article/505/3/4345/6287587?login=false)]

<img src="progetto/ref1.png" alt="y" style="width: 300px;"/>

Purtroppo, però, questa rete ha richiesto un costo computazionale troppo alto per il mio computer, pertanto non è stato possibile completarne l'esecuzione. Ne viene comunque dato un esempio, che mi riserverò di proporre per completezza di analisi. La necessità di scrivere questa rete trova giustificazione nel fatto che ancora si confondono (sebbene come atteso) le ultime due classi.

## CONCLUSIONI

I modelli messi in campo per classificare le stelle per mezzo delle loro curve di luce simulate nelle cinque classi in cui si suddividono hanno dato dei buoni risultati, riuscendo a mantenere un'accuratezza superiore all'85%. Anche laddove dalla matrice di confusione si riscontrano predizioni sbagliate, queste appaiono in posizioni che era facile prevedere sarebbero andate peggio, pertanto in linea con quanto atteso. Tuttavia, si osserva che vi sono ancora dei buoni di miglioramento che possono essere sintetizzati in questi passaggi:

- compilazione rete RNN e rete CNN+RNN per riuscire meglio a discriminare le ultime due classi;
- utilizzo di un dataset da 100,000 curve di luce (recentemente prodotto) più rappresentativo di ciascuna classe;
- test con dati reali.

In [ ]:
full_df_ANN = pd.read_csv('interp_lc_5sem.csv')
X = full_df_ANN.drop(columns=['target']).values
y = full_df_ANN['target'].values

scaler = StandardScaler()
X_scal = scaler.fit_transform(X)

y_enc = pd.get_dummies(y, dtype=int)

X_train, X_test, y_train, y_test = train_test_split(X_scal, y_enc, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

y_train_lab = np.argmax(y_train.values, axis=1)

smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train_lab)

y_train_res = pd.get_dummies(y_train_res)

X_train_res = X_train_res.reshape((X_train_res.shape[0], X_train_res.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = Sequential([
        GRU(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train_res.shape[1], 1)),
        Dropout(0.2),
        BatchNormalization(),

        GRU(128, return_sequences=False, kernel_regularizer=regularizers.l2(0.01)),
        Dropout(0.2),
        BatchNormalization(),

        Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        Dropout(0.2),

        Dense(5, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
checkpoint = ModelCheckpoint('best_model_gru.keras', monitor='val_loss', save_best_only=True)

history = model.fit(
    x=X_train_res, y=y_train_res,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=25,
    callbacks=[early_stopping, lr_scheduler, checkpoint],
    verbose=2)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest Accuracy: {test_acc:.4f}')
y_pred = np.argmax(model.predict(X_test), axis=-1)

Analogamente alla RNN si può immaginare di scrivere una rete ibrida che faccia uso di entrambe le strategie (CNN+RNN) per un'analisi più precisa, ma anche qua il costo computazionale non è stato gestibile.

In [ ]:
## CNN + RNN ##

full_df_ANN = pd.read_csv('interp_lc_5sem.csv')
X = full_df_ANN.drop(columns=['target']).values
y = full_df_ANN['target'].values

scaler = StandardScaler()
X_scal = scaler.fit_transform(X)

y_enc = pd.get_dummies(y, dtype=int)

X_train, X_test, y_train, y_test = train_test_split(X_scal, y_enc, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

y_train_lab = np.argmax(y_train.values, axis=1)

smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train_lab)

y_train_res = pd.get_dummies(y_train_res)

X_train_res = X_train_res.reshape((X_train_res.shape[0], X_train_res.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definizione del modello CNN+RNN
model = Sequential([
    Conv1D(filters=32,kernel_size=3, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train_res.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),

    Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),

    GRU(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),
    BatchNormalization(),

    GRU(64, return_sequences=False, kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.4),
    BatchNormalization(),

    Flatten(),

    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.2),

    Dense(5, activation='softmax')
])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
checkpoint = ModelCheckpoint('best_model_cnn_lstm.keras', monitor='val_loss', save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    x=X_train_res, y=y_train_res,
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[early_stopping, lr_scheduler, checkpoint],
    verbose=2)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest Accuracy: {test_acc:.4f}')

y_pred = np.argmax(model.predict(X_test), axis=-1)

## APPENDICE
Qui vengono conservati dei pezzi di codice scritto ma alla fine non utilizzato per idee/strategie poi non utilizzate

In [ ]:
'''
### QUESTO BLOCCO SERVE A IDENTIFICARE SEPARATAMENTE GLI ARRAY DEI SEMESTRI OSSERVATIVI
### utile nel caso di rebinning
rest_indices = np.where(seasonal_mask==0)[0]

#funzione per trovare i blocchi continui di osservazione attiva
def find_active_blocks(seasonal_array):
    active_indices = np.where(seasonal_array == 1)[0]
    active_blocks = []
    start_idx = active_indices[0]
    
    for i in range(1, len(active_indices)):
        if active_indices[i] != active_indices[i - 1] + 1:
            active_blocks.append((start_idx, active_indices[i - 1]))
            start_idx = active_indices[i]
    active_blocks.append((start_idx, active_indices[-1]))
    
    return active_blocks

#trova e stampa i blocchi attivi
active_blocks = find_active_blocks(seasonal_mask)
print(active_blocks)
'''

[(0, 181), (364, 545), (728, 909)]


In [ ]:
'''
### altro tentativo che avevo fatto per fittare le curve ###
#ciò sarebbe stato utile o per trovare grandezze fisiche nel pattern (periodo, fase, ampiezza)
#oppure come metodo di interpolazione alternativo ad 'inter1d'

y_curve = interpolated_curves[n] # curva di luce n-esima

degree = 3 #grado della funzione da fittare
coefficients = np.polyfit(x_filt, y_curve[seasonal_mask], degree)
poly_func = np.poly1d(coefficients) #funzione polinomiale dai coefficienti

# creo un array per tracciare la curva polinomiale
x_fit = np.linspace(np.min(x_filt), np.max(x_filt), 100)
y_fit = poly_func(x_fit)

# plot
plt.figure(figsize=(10, 6))
plt.scatter(x_filt, y_curve[seasonal_mask], s=2, color='#ff7f0e', label='Curva di luce originale')
plt.plot(x_fit, y_fit, color='red', label=f'Polinomio di grado {degree}')
plt.ylim(-1.2, 1.2)
plt.xlabel('Tempo')
plt.ylabel('Intensità')
plt.title(f'Curva di Luce Simulata {n+1} con polyfit di grado {degree}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
'''

In [ ]:
'''
### ESPLORAZIONE IPERPARAMETRI PER NN
from tensorflow.keras.utils import to_categorical
from kerastuner import HyperModel, RandomSearch

#Keras Tuner
class MyHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(layers.Input(shape=(X_train.shape[1],)))

        # Aggiungi i layer densi
        for i in range(hp.Int('num_layers', 2, 6)):  # Numero di layer densi
            units = hp.Int(f'units_{i}', min_value=64, max_value=512, step=64)
            model.add(Dense(units, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
            model.add(Dropout(hp.Float('dropout_' + str(i), 0.2, 0.4, step=0.1)))

        model.add(Dense(5, activation='softmax'))  # Output per 5 classi

        # Compila il modello
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

tuner = RandomSearch(
    MyHyperModel(),
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='helloworld7'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

tuner.search(X_train, y_train, 
             validation_data=(X_val, y_val), 
             epochs=10, 
             callbacks=[early_stopping, lr_scheduler, checkpoint], 
             verbose=2)

#ritorna il miglior modello
best_model = tuner.get_best_models(num_models=1)[0]

#stampa la migliore combinazione di parametri
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Migliore combinazione di parametri:")
print(f"Numero di layer: {best_hyperparameters.get('num_layers')}")
for i in range(best_hyperparameters.get('num_layers')):
    print(f"Layer {i + 1}: Units = {best_hyperparameters.get(f'units_{i}')}, Dropout = {best_hyperparameters.get(f'dropout_{i}')}")

test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest Accuracy: {test_acc:.4f}')

y_pred = np.argmax(best_model.predict(X_test), axis=-1)
'''

In [ ]:
'''
#numero di alberi ottimale per random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

#array per i numeri di alberi da testare
n_estimators = np.arange(1, 201, 10)  # Testiamo da 1 a 200 alberi con incrementi di 10
scores = []  # Per memorizzare le performance

#ciclo per calcolare le performance
for n in n_estimators:
    clf = RandomForestClassifier(n_estimators=n, random_state=42, n_jobs=-1)
    # Eseguiamo la validazione incrociata e calcoliamo la media dei punteggi
    score = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')  # Usando accuracy come metrica
    scores.append(score.mean())

plt.figure(figsize=(10, 6))
plt.plot(n_estimators, scores, marker='o')
plt.title('Performance del Random Forest in funzione del numero di alberi')
plt.xlabel('Numero di Alberi')
plt.ylabel('Accuratezza Media (CV)')
plt.grid(True)
plt.xticks(n_estimators)
plt.show()
'''